In [1]:
import sys
import os
# add path to the parent directory (note this is a notebook)
sys.path.append('..')
from pprint import pprint
import rlrom
import rlrom.utils as hf


/home/alex/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
repo_id='sb3/ppo-Pendulum-v1'
filename='ppo-Pendulum-v1.zip'
env_name = 'Pendulum-v1'
model = hf.load_ppo_model(env_name, repo_id)


== CURRENT SYSTEM INFO ==
- OS: Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35 # 1 SMP Thu Jan 11 04:09:03 UTC 2024
- Python: 3.10.12
- Stable-Baselines3: 2.2.1
- PyTorch: 2.2.1+cu121
- GPU Enabled: True
- Numpy: 1.26.4
- Cloudpickle: 3.0.0
- Gymnasium: 0.29.1
- OpenAI Gym: 0.26.2

== SAVED MODEL SYSTEM INFO ==
OS: Linux-5.4.0-110-generic-x86_64-with-debian-bullseye-sid #124-Ubuntu SMP Thu Apr 14 19:46:19 UTC 2022
Python: 3.7.12
Stable-Baselines3: 1.5.1a6
PyTorch: 1.11.0+cpu
GPU Enabled: False
Numpy: 1.21.6
Gym: 0.21.0



/home/alex/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


In [3]:
tester = rlrom.RLModelTester(env_name, model,render_mode='human')
print(tester.env_name)

Pendulum-v1


In [5]:
tester.test_random()

error: display Surface quit

In [5]:
import pandas as pd
df_signals = tester.get_df_signals()
print(df_signals.head())

   time  torque  cos_theta  sin_theta  theta_dot    reward
0   1.0    -2.0   0.997243   0.074209   0.900927 -0.090684
1   2.0    -2.0   0.994270   0.106902   0.656584 -0.058582
2   3.0    -2.0   0.991698   0.128588   0.436761 -0.039703
3   4.0    -2.0   0.990131   0.140142   0.233202 -0.029208
4   5.0    -2.0   0.989861   0.142038   0.038308 -0.024459


In [6]:
signals_string = """
torque, 

cos_theta,sin_theta, theta_dot, 
reward
"""
signals_layout = hf.get_layout_from_string(signals_string)
print(signals_layout)

[['torque'], ['cos_theta', 'sin_theta', 'theta_dot'], ['reward']]


In [23]:
# using plotly subplots, creates a figure with subplots for each signal list in signals_layout
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#fig.add_trace(go.Scatter(x=df_signals["time"], y=df_signals[0]), row=1, col=1)
fig = make_subplots(rows=len(signals_layout), cols=1)
for i, signal_list in enumerate(signals_layout):
    for signal in signal_list:
        fig.add_trace(go.Scatter(x=df_signals["time"], y=df_signals[signal], name=signal), row=i+1, col=1)
fig.show()



In [88]:
import stlrom
stl_driver = stlrom.STLDriver()

for trace_state in tester.trace:
    stl_driver.add_sample(tester.get_signal_state(trace_state))

stl_string  = tester.get_signal_string()+""" 
mu := reward[t] > 0 
phi := ev_[0,100] mu
"""

stl_driver.parse_string(stl_string)

True

In [89]:

import re 
# regular expression of beginning of line, then whitespaces followed by a valid variable id, followed by whitepaces, followed by :=
regex = r"^\s*\b([a-zA-Z_][a-zA-Z0-9_]*)\b\s*:="

# find all variable id in the stl_string
formulas = re.findall(regex, stl_string, re.MULTILINE)
print(formulas)


['mu', 'phi']


In [90]:
# get row i in df_signals as an array
i = 0
print(df_signals.iloc[i].values)

[ 1.         -2.          0.99724269  0.07420918  0.90092736 -0.09068416]


In [ ]:
# get number of rows in df_signals
n = len(df_signals)
print(range(n))
for i in range(n):
    print(df_signals.iloc[i].values)

In [95]:
df_robs = pd.DataFrame(columns=["time"] + formulas, dtype=float)

# fill first row of df_robs with 0
# df_robs.loc[0] = [0] + [0]*len(formulas)




In [102]:

for idx in range(len(df_signals)):    
    time = df_signals["time"][idx]
    values = []
    for idx_formula in range(len(formulas)):
        v = stl_driver.get_online_rob(formulas[idx_formula], time)[0]
        values.append(v)
    df_robs.loc[idx] = [time] + values

print(df_robs.head(1))

   time        mu      phi
0   1.0 -0.090684 -0.00347


In [101]:
# check if roro is a column of df_robs
print("roro" in df_robs.columns)

AttributeError: 'DataFrame' object has no attribute 'clear'

In [ ]:

for idx in range(len(df_signals)):
    for idx_formula in range(len(formulas)):
       print(idx_formula)
       time = df_signals["time"]
       df_robs["time"] = time
       df_robs[formulas[idx_formula]] = stl_driver.get_online_rob(formulas[idx_formula], time)[0]